In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

drev_lots = pd.read_csv("../../web/exports_igpgascogne/drev_lots.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Identifiant': 'str', 'Campagne': 'str', 'Siret Opérateur': 'str', 'Code postal Opérateur': 'str'}, low_memory=False)
etablissements = pd.read_csv("../../web/exports_igpgascogne/etablissements.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Login': 'str', 'Identifiant etablissement': 'str'}, index_col=False, low_memory=False)
societe = pd.read_csv("../../web/exports_igpgascogne/societe.csv", encoding="iso8859_15", delimiter=";", decimal=",", dtype={'Identifiant': 'str', 'Téléphone' :'str', 'Téléphone portable': 'str'}, index_col=False, low_memory=False)
lots = pd.read_csv("../../web/exports_igpgascogne/lots.csv", encoding="iso8859_15", delimiter=";", decimal=",", index_col=False, low_memory=False)


In [ ]:
def createCSVByCampagne(campagne,drev_lots,etablissements,societe,lots):
    drev_lots = drev_lots.query("Campagne == @campagne");
    drev_lots = drev_lots.groupby(['Identifiant','Famille','CVI Opérateur','Siret Opérateur','Nom Opérateur','Adresse Opérateur','Code postal Opérateur','Commune Opérateur','Email Operateur','Appellation'])[["Volume"]].sum()
    drev_lots = drev_lots.reset_index()
    
    drev_lots = pd.merge(drev_lots, etablissements, how='left',left_on=["Identifiant"], right_on=["Identifiant etablissement"],suffixes=("", " etablissement"))
    drev_lots = pd.merge(drev_lots, societe , how ="left" , left_on ="Login", right_on ="Identifiant",suffixes=("", " societe"))
    
    conforme = "Conforme"
    rep_conforme = "Réputé conforme"
    lots = lots.query("Campagne == @campagne");
    
    lots = lots.rename(columns = {'Statut de lot': 'Statut_de_lot'})

    lots = lots.query("Statut_de_lot != @conforme & Statut_de_lot != @rep_conforme");
           
    lots = lots.groupby(['Id Opérateur','Nom Opérateur','Appellation'])[["Volume"]].sum()
    
    drev_lots = pd.merge(drev_lots, lots , how='left', left_on = ["Identifiant",'Nom Opérateur','Appellation'], right_on = ["Id Opérateur",'Nom Opérateur','Appellation'],suffixes=("", " lots"))
    
    drev_lots =  drev_lots.rename(columns = {'Volume': 'Volume revendiqué','Volume lots':'Volume en instance de conformité'})
    #print(drev_lots.columns)
    
    #colonnes à avoir dans le csv final
    drev_lots = drev_lots[['Identifiant','Appellation','Nom Opérateur','Raison sociale','Siret Opérateur', 'CVI Opérateur','Adresse societe','Adresse 2 societe','Adresse 3 societe','Code postal societe', 'Commune societe','Email','Téléphone','Téléphone portable', 'Fax societe','Volume revendiqué','Volume en instance de conformité']]
    drev_lots.to_csv('../../web/exports/igp_stats_droit_inao_operateurs_redevable'+campagne+".csv", encoding="iso8859_15", sep=";", decimal=",")
       
    
createCSVByCampagne("2019-2020",drev_lots,etablissements,societe,lots)
createCSVByCampagne("2020-2021",drev_lots,etablissements,societe,lots)
